In [22]:
import pandas as pd
import praw
import yaml
from praw.models import MoreComments
from yaml.loader import SafeLoader

with open('../config-personal.yml') as f:
    config = yaml.load(f, Loader=SafeLoader)

reddit = praw.Reddit(
    client_id=config['reddit']['client_id'],
    client_secret=config['reddit']['secret'],
    user_agent=f"testscript by u/{config['reddit']['user_name']}",
)

df = pd.read_csv('../data/wallstreetbets.zip')


In [12]:
df.info( )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53187 entries, 0 to 53186
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      53187 non-null  object 
 1   score      53187 non-null  int64  
 2   id         53187 non-null  object 
 3   url        53187 non-null  object 
 4   comms_num  53187 non-null  int64  
 5   created    53187 non-null  float64
 6   body       24738 non-null  object 
 7   timestamp  53187 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 3.2+ MB


In [19]:
first_row = df.head(1).iloc[0,:]
first_row

title        It's not about the money, it's about sending a...
score                                                       55
id                                                      l6ulcx
url                            https://v.redd.it/6j75regs72e61
comms_num                                                    6
created                                           1611862661.0
body                                                       NaN
timestamp                                  2021-01-28 21:37:41
Name: 0, dtype: object

In [20]:
submission = reddit.submission(first_row.id)

In [26]:
comment_ls = []
for top_level_comment in submission.comments:
    if isinstance(top_level_comment, MoreComments):
        continue
    comment_ls.append(top_level_comment.body)

In [28]:
comment_ls[0:5]

['yeah just hope everyone deletes their 420 limits. 1000 IS NOT A JOKE',
 'The video is pretty but I cant fucking read',
 'It’s not about the money, it’s about a shitload of money.',
 "$10000 here we come boys and gals. Let's drain their bank accounts like they've done to us all these years.",
 '[deleted]']

In [31]:
comment_dict = {'id_col': [], 'comments': []}
id_ls = df.loc[:,'id'].tolist()
id_test = id_ls[0:4]

for post_id in id_test:
    comment_ls = []
    post = reddit.submission(post_id)
    for top_level_comment in post.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        comment_ls.append(top_level_comment.body)

    comment_dict['id_col'].extend([post_id]*len(comment_ls))
    comment_dict['comments'].extend(comment_ls)

In [34]:
pd.DataFrame(comment_dict).head(5)

,id_col,comments
0,l6ulcx,yeah just hope everyone deletes their 420 limi...
1,l6ulcx,The video is pretty but I cant fucking read
2,l6ulcx,"It’s not about the money, it’s about a shitloa..."
3,l6ulcx,$10000 here we come boys and gals. Let's drain...
4,l6ulcx,[deleted]


It works! Now do it for all comments.